In [5]:
df = spark.sql("SELECT * FROM bing_lake_db_dev.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c8ca9c22-78e6-421f-a85e-7f5a953bcdb4)

In [1]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 3, Finished, Available, Finished)

In [2]:
model = (
    AnalyzeText()
    .setTextCol("description")
    .setKind("SentimentAnalysis")
    .setOutputCol("response")
    .setErrorCol("error")
)

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 4, Finished, Available, Finished)

In [6]:
result = model.transform(df)

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 8, Finished, Available, Finished)

In [7]:
display(result)

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 24ff0820-1404-44a5-95dc-e237f758ace4)

In [8]:
from pyspark.sql.functions import col
sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 10, Finished, Available, Finished)

In [9]:
display(sentiment_df)

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 51f06713-7907-48cc-a907-d35c0d90d522)

In [10]:
sentiment_df = sentiment_df.drop("error", "response")
display(sentiment_df)

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 882381f1-8ead-4207-909e-021411875d16)

In [12]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = "bing_lake_db_dev.tbl_sentiment_analysis"
    sentiment_df.write.format("delta").saveAsTable(table_name)

except:
    print("Table Already Exists")

    sentiment_df.createOrReplaceTempView("vw_sentiment_df")

    spark.sql(f"""
                MERGE INTO {table_name} target_table
                USING vw_sentiment_df source_view

                ON source_view.url = target_table.url

                WHEN MATCHED AND

                source_view.title <> target_table.title OR
                source_view.description <> target_table.description OR
                source_view.category <> target_table.category OR
                source_view.image <> target_table.image OR
                source_view.provider <> target_table.provider OR
                source_view.datePublished <> target_table.datePublished

                THEN UPDATE SET *
                WHEN NOT MATCHED THEN INSERT * 
    """)

StatementMeta(, 9cdb358a-f8c1-49db-b910-2420f38b95eb, 14, Finished, Available, Finished)

Table Already Exists
